In [ ]:
 # 导入re模块，用于正则表达式操作
import re

# 定义文件路径和打开文件
channel_id = 'channel_id/UCghLs6s95LrBWOdlZUCH4qw.txt'
with open(channel_id, 'r') as channel:
    # 读取文件内容，并赋值给xml变量
    xml = channel.read()

# 定义正则表达式模式，用于匹配视频ID
id_pattern = re.compile(r'(?<=<id>yt:video:)[A-Za-z0-9_-]{11}(?=</id>)')

# 在xml内容中搜索视频ID，并赋值给video_ids变量
video_ids = id_pattern.findall(xml)

# 输出匹配到的视频ID
print(video_ids)

In [ ]:
import requests
import xml.etree.ElementTree as ET
import time

# 根据需要修改以下变量
url = "https://www.youtube.com/feeds/videos.xml?channel_id=UCghLs6s95LrBWOdlZUCH4qw"
pushkey = "PDU18136TvdVYY5K3UagCr7zYKZozIqT3C5VrCScs"

# 初始化变量
last_updated = None
update_interval = 5 * 60  # 更新间隔（单位：秒）

while True:
    # 检查更新
    response = requests.get(url)
    if response.status_code == 200:
        content = response.content
        tree = ET.fromstring(content)
        pub_date = tree.find("./{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}published").text
        if pub_date != last_updated:
            # 触发 API 通知
            url_payload = f"https://api2.pushdeer.com/message/push?pushkey={pushkey}&text=YouTube更新"
            response = requests.get(url_payload)
            
            # 更新上次更新时间
            last_updated = pub_date
            print("已检测到更新，已发送通知")
        else:
            print("没有更新")
    else:
        print("请求失败")
    
    # 等待下次更新
    time.sleep(update_interval)

In [ ]:
import datetime

log = "测试"

# 获取当前的具体时间
current_time = datetime.datetime.now()

# 格式化输出，只保留年月日时分秒
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

# 打印当前的具体时间（精确到秒）
print(formatted_time,log)

try:
    # 打开文件，在末尾追加内容
    with open("log.txt", "a") as file:
        file.write(formatted_time + log + "\n")

except FileNotFoundError:
    # 文件不存在的处理逻辑
    with open("log.txt", "w") as file:
        file.write(formatted_time + log + "\n")

In [ ]:
import datetime

def write_log(log):
    # 获取当前的具体时间
    current_time = datetime.datetime.now()

    # 格式化输出，只保留年月日时分秒
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

    # 打印当前的具体时间（精确到秒）
    print(formatted_time, log)

    try:
        # 打开文件，在末尾追加内容
        with open("log.txt", "a") as file:
            file.write(formatted_time + " " + log + "\n")

    except FileNotFoundError:
        # 文件不存在的处理逻辑
        with open("log.txt", "w") as file:
            file.write(formatted_time + " " + log + "\n")


write_log("nihao")

In [ ]:
write_log("ni hao")

In [ ]:
import datetime

def write_log(log):
    # 获取当前的具体时间
    current_time = datetime.datetime.now()

    # 格式化输出，只保留年月日时分秒
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

    # 打开文件，并读取原有内容
    try:
        with open("log.txt", "r") as file:
            contents = file.read()
    except FileNotFoundError:
        contents = ""

    # 将新的日志内容添加在原有内容之前
    new_contents = formatted_time + " " + log + "\n" + contents

    # 将新的日志内容写入文件
    with open("log.txt", "w") as file:
        file.write(new_contents)

    # 打印当前的具体时间（精确到秒）
    print(formatted_time, log)


In [ ]:
write_log("ni hao")

In [ ]:
import subprocess

library = "rangehttpserver"

def is_library_installed():
    try:
        result = subprocess.run([library , '--version'], capture_output=True, text=True)
        if result.returncode == 0:
            return True
        else:
            return False
    except FileNotFoundError:
        return False

def install_library():
    try:
        result = subprocess.run(['pip', 'install', library], capture_output=True, text=True)
        if result.returncode == 0:
            return True
        else:
            return False
    except FileNotFoundError:
        return False

# 更新yt-dlp
def update_library():
    try:
        result = subprocess.run(['pip', 'install', '--upgrade', library], capture_output=True, text=True)
        if result.returncode == 0:
            return True
        else:
            return False
    except FileNotFoundError:
        return False

# 检查是否已安装yt-dlp
if is_library_installed():
    print(library + "已安装")
else:
    print(library + "未安装")

# 安装或更新yt-dlp
if is_library_installed():
    if update_library():
        print(library + "更新成功")
    else:
        print(library + "更新失败")
else:
    if install_library():
        print(library + "安装成功")
    else:
        print(library + "安装失败")


In [ ]:
import subprocess
# 安装库模块
def library_install(library):
    # 检查库是否已安装
    def is_library_installed():
        try:
            result = subprocess.run([library , '--version'], capture_output=True, text=True)
            if result.returncode == 0:
                return True
            else:
                return False
        except FileNotFoundError:
            return False
    # 如果库未安装，则尝试安装
    def install_library():
        try:
            result = subprocess.run(['pip', 'install', library], capture_output=True, text=True)
            if result.returncode == 0:
                return True
            else:
                return False
        except FileNotFoundError:
            return False
    # 如果库已安装，则尝试更新
    def update_library():
        try:
            result = subprocess.run(['pip', 'install', '--upgrade', library], capture_output=True, text=True)
            if result.returncode == 0:
                return True
            else:
                return False
        except FileNotFoundError:
            return False
    # 检查库是否已安装
    if is_library_installed():
        print(library + "已安装")
    else:
        print(library + "未安装")
    # 如果库已安装，则尝试更新
    if is_library_installed():
        if update_library():
            print(library + "更新成功")
        else:
            print(library + "更新失败")
    else:  # 如果库未安装，则尝试安装
        if install_library():
            print(library + "安装成功")
        else:
            print(library + "安装失败")

In [ ]:
library_install("yt-dlp")

In [ ]:
import requests
url = "https://www.youtube.com/feeds/videos.xml?channel_id=UCBR8-60-B28hp2BmDPdntc"
response = requests.get(url)  # 发送GET请求获取网页内容
content = response.text 

print (content)



import re
# 判断频道id是否正确
pattern_youtube404 = r"Error 404"  # 设置要匹配的正则表达式模式
if not re.search(pattern_youtube404, content):
    print("文本中不包含 'Error 404 (Not Found)!!'")
else:
    print("不正确")


In [ ]:
{
        "YouTube": {
            "update_size": 1,
            "id": "UCBR8-60-B28hp2BmDPdntcQ",
            "title": "YouTube",
            "quality": "720",
            "last_size": 1,
            "media": "m4a"
        },
        "科学火箭叔": {
            "update_size": 15,
            "id": "UC8dD8v_uIA7aRzpqaCoIHKg",
            "title": "科学火箭叔",
            "quality": "1080",
            "last_size": 20,
            "media": "mp4"
        }
    }
#如何自动生成为：
{"UCBR8-60-B28hp2BmDPdntcQ": "YouTube","UC8dD8v_uIA7aRzpqaCoIHKg": "科学火箭叔"}

In [ ]:
import yt_dlp

def video_duration(video_url):
    try:
        # 创建yt-dlp实例
        ydl = yt_dlp.YoutubeDL({'ignoreerrors': True})
        if info_dict := ydl.extract_info(
            f"https://www.youtube.com/watch?v={video_url}", download=False
        ):
            # 获取视频时长并回显
            return info_dict.get('duration')
    except Exception as e:
        print((f"{video_url}下载失败，错误信息：{str(e)}").replace("ERROR: ", "").replace(f"{video_url}: ", ""))

print(video_duration("HcMr1gG0iho"))

In [ ]:
import subprocess
import re

def get_duration_ffprobe(file_path):
    try:
        command = [
            "ffprobe",
            "-i", file_path,
            "-show_entries", "format=duration",
            "-v", "error",
            "-of", "default=noprint_wrappers=1:nokey=1"
        ]
        output = subprocess.check_output(command, stderr=subprocess.STDOUT).decode("utf-8")
        return float(output)
    except subprocess.CalledProcessError as e:
        print(f"Error: {e.output}")
        return None

file_path = "UCghLs6s95LrBWOdlZUCH4qw/HcMr1gG0iho.m4a"  # 替换为您的文件路径
duration = get_duration_ffprobe(file_path)
if duration is not None:
    print(f"Duration: {duration} seconds")
    
print(round(get_duration_ffprobe("UCghLs6s95LrBWOdlZUCH4qw/HcMr1gG0iho.m4a")))


In [ ]:
def c():
    a = 1
    b = 2
    return a,b
print(c())

In [ ]:
from pytube import YouTube

try:
    yt = YouTube(f"https://www.youtube.com/watch?v=YKJQc_Nf3QQ")    
    # 检查视频信息是否成功获取
    if yt.vid_info:
        # 获取视频时长（以秒为单位）
        duration = yt.length
        # 打印结果
        print(f"视频时长: {duration} 秒")
except Exception as e:
    print("发生其他错误:", e)

In [ ]:
from pytube import YouTube

video_url = "https://www.youtube.com/watch?v=lq6ZIbZ1Xrg"

# 创建YouTube对象
yt = YouTube(video_url)

# 获取视频标题
title = yt.title

# 获取视频时长（以秒为单位）
duration = yt.length

# 获取视频作者
author = yt.author

# 获取视频描述
description = yt.description

# 获取视频发布日期
publish_date = yt.publish_date

# 打印结果
print(f"视频标题: {title}")
print(f"视频时长: {duration} 秒")
print(f"视频作者: {author}")
print(f"视频描述: {description}")
print(f"视频发布日期: {publish_date}")

In [ ]:
print(YouTube(video_url).length)

In [ ]:
channel_list = {
    "科学火箭叔": "UC8dD8v_uIA7aRzpqaCoIHKg",
    "stone pistol": "UCmOVUFkam162EJ48KfQaIyg",
    "奇异博士说": "UCVaMMNaYvjmskreBR4dM7rg",
    "科技美学": "UCAC204niCVvtDRDFYSiK7Cg",
    "stone记": "UCghLs6s95LrBWOdlZUCH4qw",
    "Linvo說宇宙": "UCjHCBI7CNqQwzuaiDtCAQmA",
    "四季妈妈—小龙": "UC3elWG-AxVvIF2_xPQb1rkQ",
    "Discovery With Andy": "UC6lYCOXS4Rxo9xpKS1My32A",
    "冲浪普拉斯": "UCZNxe_9FQAK2Z3suwaZlwpQ",
    "老高與小茉 Mr & Mrs Gao": "UCMUnInmOkrWN4gof9KlhNmQ",
    "脑洞乌托邦": "UC2tQpW0dPiyWPebwBSksJ_g",
    "啾啾鞋": "UCIF_gt4BfsWyM_2GOcKXyEQ",
    "幻海航行--science fiction": "UCp1nO1bgVwks9b5EhKQGVag",
    "红酒汤姆一世": "UCJT6M5j-HT-76ts0eTzZtJw",
    "X調查": "UCOyshL6rKK1GqwoEfy_ehBg",
    "海伦子Hellen": "UCneG8p_dRmZZ_xFFCv7jfJw",
    "你可敢信": "UC2lbb8vjQR64C6gHqD0_4OA",
    "曉涵哥來了": "UCvoBl4rnVsetDKA_Tdk-jeA",
    "自说自话的总裁": "UCgo_-fjJxnLwwwq5dSY72rg",
    "林亦LYi": "UC4dtpugIYK56S_7btf5a-iQ",
    "小乌副频道": "UCQvvVtUqtGxqSLixdeG75Uw"
}

output = {
    "YouTube": {
        "update_size": 15,
        "id": "UCBR8-60-B28hp2BmDPdntcQ",
        "title": "YouTube",
        "quality": "1080",
        "last_size": 30,
        "media": "m4a"
    }
}

for channel, channelId in channel_list.items():
    output[channel] = {
        "update_size": 15,
        "id": channelId,
        "title": channel,
        "quality": "1080",
        "last_size": 30,
        "media": "m4a"
    }

print(output)


In [ ]:
my_dict = {'key1': 'value1', 'key1': 'value2', 'key2': 'value3'}



In [ ]:
def dict_exclude_repeat(dict):
    dict_new = {}
    for key, value in dict.items():
        if key not in dict_new:
            dict_new[key] = value
    return dict_new
            
print(dict_exclude_repeat(my_dict))

In [ ]:
def dict_exclude_repeat(my_dict):
    dict_new = {}
    for key, value in my_dict.items():
        if key not in dict_new:
            dict_new[key] = value
        print(key+value)
    return dict_new

my_dict = {'key1': 'value1', 'key2': 'value2', 'key3': 'value3', 'key2': 'value4', 'key3': 'value5'}

new_dict = dict_exclude_repeat(my_dict)
print(new_dict)
print(my_dict)

In [ ]:
import yt_dlp

def video_duration(video_url):
    try:
        # 创建yt-dlp实例
        ydl = yt_dlp.YoutubeDL({'ignoreerrors': True})
        if info_dict := ydl.extract_info(
            f"https://www.youtube.com/watch?v={video_url}", download=False
        ):
            # 获取视频时长并回显
            return info_dict.get('duration')
    except Exception as e:
        print((f"{video_url}下载失败，错误信息：{str(e)}").replace("ERROR: ", "").replace(f"{video_url}: ", ""))



import subprocess
import re

def get_duration_ffprobe(file_path):
    try:
        command = [
            "ffprobe",
            "-i", file_path,
            "-show_entries", "format=duration",
            "-v", "error",
            "-of", "default=noprint_wrappers=1:nokey=1"
        ]
        output = subprocess.check_output(command, stderr=subprocess.STDOUT).decode("utf-8")
        return round(float(output))
    except subprocess.CalledProcessError as e:
        print(f"Error: {e.output}")
        return None


if video_duration("HcMr1gG0iho") == get_duration_ffprobe("UCghLs6s95LrBWOdlZUCH4qw/HcMr1gG0iho.m4a"):
    print("True")



In [ ]:
a=0
b=2
d = None
while a < b and d:
    a =+ 1
    print(a)

In [ ]:
import xml.etree.ElementTree as ET

# 创建元素对象
rss = ET.Element('rss')
rss.set('version', '2.0')
rss.set('xmlns:itunes', 'http://www.itunes.com/dtds/podcast-1.0.dtd')

channel = ET.SubElement(rss, 'channel')

title = ET.SubElement(channel, 'title')
title.text = 'YouTube'

link = ET.SubElement(channel, 'link')
link.text = 'https://m.youtube.com'

description = ET.SubElement(channel, 'description')
description.text = '在YouTube 上畅享您喜爱的视频和音乐，上传原创内容并与亲朋好友和全世界观众分享您的视频。'

category = ET.SubElement(channel, 'category')
category.text = 'TV & Film'

generator = ET.SubElement(channel, 'generator')
generator.text = 'Podsync generator (support us at https://github.com/mxpv/podsync)'

language = ET.SubElement(channel, 'language')
language.text = 'en-us'

lastBuildDate = ET.SubElement(channel, 'lastBuildDate')
lastBuildDate.text = 'Wed, 30 Aug 2023 05:39:32 +0800'

pubDate = ET.SubElement(channel, 'pubDate')
pubDate.text = 'Sun, 24 Apr 2005 11:20:54 +0800'

image = ET.SubElement(channel, 'image')
image_url = ET.SubElement(image, 'url')
image_url.text = 'https://static.vecteezy.com/system/resources/previews/003/399/771/original/youtube-icon-editorial-free-vector.jpg'
image_title = ET.SubElement(image, 'title')
image_title.text = 'YouTube'
image_link = ET.SubElement(image, 'link')
image_link.text = 'https://m.youtube.com'

itunes_author = ET.SubElement(channel, 'itunes:author')
itunes_author.text = 'YouTube'

itunes_subtitle = ET.SubElement(channel, 'itunes:subtitle')
itunes_subtitle.text = 'YouTube'

itunes_summary = ET.SubElement(channel, 'itunes:summary')
itunes_summary.text = '在YouTube 上畅享您喜爱的视频和音乐，上传原创内容并与亲朋好友和全世界观众分享您的视频。'

itunes_image = ET.SubElement(channel, 'itunes:image')
itunes_image.set('href', 'https://static.vecteezy.com/system/resources/previews/003/399/771/original/youtube-icon-editorial-free-vector.jpg')

itunes_explicit = ET.SubElement(channel, 'itunes:explicit')
itunes_explicit.text = 'no'

itunes_category = ET.SubElement(channel, 'itunes:category')
itunes_category.set('text', 'TV & Film')

# 将XML写入文件或以其他方式使用
tree = ET.ElementTree(rss)
tree.write('example.xml', encoding='utf-8', xml_declaration=True)


In [ ]:
xml_string = ET.tostring(rss, encoding="utf-8", method="xml").decode()
print(xml_string)

In [ ]:
import time
# 获取当前时间
current_time_now = time.time()  # 获取当前时间的秒数
# 获取当前时区和夏令时信息
time_info_now = time.localtime(current_time_now)
timezone_offset_now = time.timezone
dst_offset_now = time.daylight
# 计算时区偏移量
total_offset_now = timezone_offset_now + dst_offset_now
# 构造时间字符串
formatted_time_now = time.strftime('%a, %d %b %Y %H:%M:%S %z', time_info_now)

print(formatted_time_now)



In [ ]:
import re
entry = ''' <entry>
  <id>yt:video:6hIWTOmbt8c</id>
  <yt:videoId>6hIWTOmbt8c</yt:videoId>
  <yt:channelId>UC3elWG-AxVvIF2_xPQb1rkQ</yt:channelId>
  <title>东北最宝贵的不是人参鹿茸和皮草，而是温暖的人情#爱吐槽的四季妈妈 #家庭吐槽大会</title>
  <link rel="alternate" href="https://www.youtube.com/watch?v=6hIWTOmbt8c"/>
  <author>
   <name>四季妈妈—小龙</name>
   <uri>https://www.youtube.com/channel/UC3elWG-AxVvIF2_xPQb1rkQ</uri>
  </author>
  <published>2023-09-03T02:06:11+00:00</published>
  <updated>2023-09-03T08:16:57+00:00</updated>
  <media:group>
   <media:title>东北最宝贵的不是人参鹿茸和皮草，而是温暖的人情#爱吐槽的四季妈妈 #家庭吐槽大会</media:title>
   <media:content url="https://www.youtube.com/v/6hIWTOmbt8c?version=3" type="application/x-shockwave-flash" width="640" height="390"/>
   <media:thumbnail url="https://i3.ytimg.com/vi/6hIWTOmbt8c/hqdefault.jpg" width="480" height="360"/>
   <media:description>东北最宝贵的不是人参鹿茸和皮草，而是温暖的人情#爱吐槽的四季妈妈 #家庭吐槽大会</media:description>
   <media:community>
    <media:starRating count="6" average="5.00" min="1" max="5"/>
    <media:statistics views="294"/>
   </media:community>
  </media:group>
 </entry>'''

print(re.search(r"(?<=<media:thumbnail url=\").+(?=\" width=\")", entry).group())

In [ ]:
a = [1,2,3,4,5,6,7,8,9,10]
b = 0
while b < 5:
    print(a[b])
    b =+ 1

In [ ]:
import re
import subprocess

def library_install(package_name):
    try:
        print(re.search(r"(?<=Version\: ).+", subprocess.run(["pip", "show", package_name], capture_output=True, text=True).stdout).group())
        is_package_installed = True
    except subprocess.CalledProcessError:
        is_package_installed = False
    if is_package_installed:
        print(f"Updating {package_name}")
        subprocess.run(["pip", "install", "--upgrade", package_name], capture_output=True, text=True)
        print(f"{package_name} 更新成功")
    else:
        print(f"Installing {package_name}")
        subprocess.run(["pip", "install", package_name], capture_output=True, text=True)
        print(f"{package_name} 安装成功")

In [ ]:
library_install("yt-dlp")

In [ ]:
f = ""
if not f:
    print("2")

In [ ]:
# 格式化时间模块
def time_format(duration):
    if duration is None:
        return "Unknown"
    duration = int(duration)
    hours, remaining_seconds = divmod(duration, 3600)
    minutes = remaining_seconds // 60
    remaining_seconds = remaining_seconds % 60
    if hours > 1:
        return '{:02}:{:02}:{:02}'.format(hours, minutes, remaining_seconds)
    else:
        return '{:02}:{:02}'.format(minutes, remaining_seconds)

# 格式化字节模块
def convert_bytes(byte_size, units = None, outweigh =1024):
    if units is None:
        units = [' B', 'KB', 'MB', 'GB']
    if byte_size is None:
        byte_size = 0
    # 初始单位是字节
    unit_index = 0
    # 将字节大小除以1024直到小于1024为止
    while byte_size > outweigh and unit_index < len(units) - 1:
        byte_size /= 1024.0
        unit_index += 1
    # 格式化结果并返回
    return f"{byte_size:.2f}{units[unit_index]}"

# 下载显示模块
def show_progress(stream):
    stream = dict(stream)
    if "downloaded_bytes" in stream:
        downloaded_bytes = convert_bytes(stream['downloaded_bytes']).rjust(9)
    else:
        downloaded_bytes = " Unknow B"
    if "total_bytes" in stream:
        total_bytes = convert_bytes(stream['total_bytes'])
    else:
        total_bytes = "Unknow B"
    if stream['speed'] is None:
        speed = " Unknow B"
    else:
        speed = convert_bytes(stream['speed'], [' B', 'KiB', 'MiB', 'GiB'], 1000).rjust(9)
    if stream['status'] in ["downloading", "error"]:
        if "total_bytes" in stream:
            bar = stream['downloaded_bytes'] / stream['total_bytes'] * 100
        else:
            bar = 0
        bar = f"{bar:.1f}" if bar == 100 else f"{bar:.2f}"
        bar = bar.rjust(5)
        eta = time_format(stream['eta']).ljust(8)
        print((f"\r\033[94m{bar}%\033[0m|{downloaded_bytes}\{total_bytes}|\033[32m{speed}/s\033[0m|\033[93m{eta}\033[0m"),end="")
    if stream['status'] == "finished":
        if "elapsed" in stream:
            elapsed = time_format(stream['elapsed']).ljust(8)
        else:
            elapsed = "Unknown "
        print((f"\r100.0%|{downloaded_bytes}\{total_bytes}|\033[32m{speed}/s\033[0m|\033[97m{elapsed}\033[0m"),end="")
        print("")

In [ ]:
import yt_dlp
# 下载视频模块
def download_video(video_url, output_dir, output_format, video_website, format_code=480, output_dir_name=""):
    if output_dir_name:
        video_write_log = f"{output_dir_name}|{video_url}"
    else:
        video_write_log = video_url
    if output_format == 'm4a':
        format_out = "bestaudio[ext=m4a]/best"   # 音频编码
    else:
        format_out = f'bestvideo[ext=mp4][height<={format_code}]+bestaudio[ext=m4a]/best'  # 视频编码
    ydl_opts = {
        'outtmpl': f'{output_dir}/{video_url}.{output_format}',  # 输出文件路径和名称
        'format': f'{format_out}',  # 指定下载的最佳音频和视频格式
        "quiet": True,
        #"no_warnings": True,
        "noprogress": True,
        "progress_hooks": [show_progress],
        #"postprocessor_hooks": [format_processing],
        #"no_part": True,
        #"debug_printtraffic": False
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([f'{video_website}{video_url}'])  # 下载指定视频链接的视频
        print(f"{video_write_log}下载成功")  # 写入下载成功的日志信息
    except Exception as e:
        print((f"{video_write_log}下载失败, 错误信息：{str(e)}").replace("ERROR: ", "").replace(f"{video_url}: ", ""))  # 写入下载失败的日志信息
        return video_url
    


def format_processing(process_stream):
    if process_stream["status"] == "started":
        print(f'\rProcessing', end="")
    elif process_stream["status"] == "finished":
        print(f'\x1b[1K\rProcessed')
    return



In [ ]:
def before_download(url):
    # 在下载开始前执行的操作
    print(url)

In [ ]:
def format_processing(process_stream):
    print(process_stream)

In [ ]:
# 文本整形模块
def shap(text, text_size):
    if isinstance(text_size, int) and text_size> 0:
        text = text.strip()
        return text if len(text) > text_size else f"{text:>{text_size}}"
    else:
        text = text.strip()
        text_size = text_size.strip()
        bit = len(text_size)
        if len(text) > bit:
            return f"{text}/{text_size}"
        else:
            return f"{text:>{bit}}/{text_size}"

# 下载显示模块
def show_progress(data_stream):
    percent_str = shap(data_stream['_percent_str'], 5)
    if '_total_bytes_estimate_str' in data_stream:
        if '_downloaded_bytes_str' in data_stream:
            downloaded_bytes_str = shap(data_stream['_downloaded_bytes_str'], data_stream['_total_bytes_estimate_str'])
        else:
            downloaded_bytes_str = shap(data_stream['_total_bytes_estimate_str'], data_stream['_total_bytes_estimate_str'])
    else:
        downloaded_bytes_str = shap(data_stream['_total_bytes_str'], data_stream['_total_bytes_str'])
    speed_str = data_stream['_speed_str'].strip()
    if '_eta_str' in data_stream:
        eta_str = shap(data_stream['_eta_str'].strip(), 5)
        print((f"\r{f'{percent_str}|{downloaded_bytes_str}|{eta_str}|{speed_str}':<44}")[:44],end="")
    else:
        eta_str = data_stream['_elapsed_str'].strip()
        if eta_str[:3] == "00:":
            eta_str = eta_str[-5:]
        print((f"\r{f'{percent_str}|{downloaded_bytes_str}|{eta_str}|{speed_str}':<44}")[:44],end="")
        print(f"\r{data_stream}",end="")
        print("")
    

In [ ]:
download_video("HgIBGdBDeHs", "text", "mp4", "https://www.youtube.com/watch?v=", "480")

In [ ]:
def convert_bytes(byte_size, units = None, outweigh =1024):
    if units is None:
        units = [' B', 'KB', 'MB', 'GB']
    if byte_size is None:
        byte_size = 0
    # 初始单位是字节
    unit_index = 0
    # 将字节大小除以1024，直到小于1024为止
    while byte_size > outweigh and unit_index < len(units) - 1:
        byte_size /= 1024.0
        unit_index += 1
    # 格式化结果并返回
    return f"{byte_size:.2f}{units[unit_index]}"

In [ ]:
convert_bytes(353580)

In [ ]:
my_dict = {"ab": 1, "cd": 2, "ef": 3}

# 判断"a"是否在字典的键中
if "a" in my_dict.keys():
    print("键'a'存在于字典中")
else:
    print("键'a'不存在于字典中")

In [ ]:
# 文本整形模块
def shap_file_size(text, text_size):
    if isinstance(text_size, int) and text_size> 0:
        text = text.strip()
        return text if len(text) > text_size else f"{text:>{text_size}}"
    else:
        text = text.strip()
        text_size = text_size.strip()
        bit = len(text_size)
        if len(text) > bit:
            return f"{text}/{text_size}"
        else:
            return f"{text:>{bit}}/{text_size}"

In [ ]:
value = -1

if isinstance(value, int) and value > 0:
    print("这是一个大于零的自然数。")
else:
    print("这不是一个大于零的自然数。")


In [ ]:
my_var = {"name": "John", "age": 30}
my_var = "abc"
if type(my_var) is dict:
    print("这是一个字典。")
else:
    my_var = {"address": "123 Main St"}
    print(my_var)

In [ ]:
#  100% of   43.95MiB in 00:00:06 at 7.16MiB/s
import re
_default_template ="71.4% of ~  43.98MiB at  650.85KiB/s ETA 00:19 (frag 206/290)"
pattern = r"( of ~ )|( at )|( ETA )|( in )|( of )"
pattern_ = r"\(.+\)"
print(f"\r{re.sub(pattern_, '', re.sub(pattern, '|', _default_template))}")

In [ ]:
import os
if not os.path.exists(os.path.join("channel_id", "UC3elWG-AxVvIF2_xPQb1rkQ.txt")):
    print("文件不存在于文件夹中。")
else:
    print("文件存在于文件夹中。")

In [ ]:
print("stEing".lower())

In [10]:
import yt_dlp


# 获取媒体时长和ID模块
def video_format(video_website, video_url, media = "m4a", quality = "480"):
    fail_message = None
    def duration_and_formats(video_website, video_url):
        fail_message, duration, formats = None, None, None
        try:
            # 初始化 yt_dlp 实例, 并忽略错误
            ydl_opts = {
                'ignoreerrors': True,
                'no_warnings': True, 
                'quiet': True  # 禁止非错误信息的输出
            }
            ydl = yt_dlp.YoutubeDL(ydl_opts)
            # 使用提供的 URL 提取视频信息
            if info_dict := ydl.extract_info(
                f"{video_website}{video_url}", download = False
            ):
                # 获取视频时长并返回
                duration = info_dict.get('duration')
                formats = info_dict.get('formats')
        except Exception as e:
            fail_message = (f"\033[31m获取信息失败\033[0m\n错误信息：{str(e)}").replace("ERROR: ", "").replace(f"{video_url}: ", "")
        return fail_message, duration, formats
    yt_id_count = 0
    fail_message, duration, formats = duration_and_formats(video_website, video_url)
    while yt_id_count < 2 and (fail_message is not None or duration is None or formats is None):
        yt_id_count += 1
        print(f"|{video_url} 无法获取媒体信息, 开始第\033[34m{yt_id_count}\033[0m次重试")
        fail_message, duration, formats = duration_and_formats(video_website, video_url)
    if fail_message is None:
        if duration == "" or duration is None:
            fail_message = f"\033[31m获取信息失败\033[0m\n错误信息：无法获取媒体时长"
            if formats == "" or formats is None:
                fail_message = f"\033[31m获取信息失败\033[0m\n错误信息：无法获取媒体格式"
        duration_and_id = []
        duration_and_id.append(duration)
        # 定义条件判断函数
        def check_resolution(item):
            if "aspect_ratio" in item and (isinstance(item["aspect_ratio"], float) or isinstance(item["aspect_ratio"], int)):
                if item["aspect_ratio"] >= 1:
                    return item["height"] <= int(quality)
                else:
                    return item["width"] <= int(quality)
            else:
                return False
        def check_ext(item, media):
            if "ext" in item:
                return item["ext"] == media
            else:
                return False
        def check_vcodec(item):
            if "vcodec" in item:
                return "vp" not in item["vcodec"].lower() and "av01" not in item["vcodec"].lower()
            else:
                return False
        #获取最好质量媒体的id
        def best_format_id(formats):
            filesize_max = 0
            format_id_best = ""
            vcodec_best = ""
            for format in formats:
                if "filesize" in format and (isinstance(format["filesize"], float) or isinstance(format["filesize"], int)) and format["filesize"] > filesize_max:
                    filesize_max = format["filesize"]
                    format_id_best = format["format_id"]
                    vcodec_best = format["vcodec"]
            return format_id_best, vcodec_best
    if fail_message is None:
        # 进行筛选
        formats_m4a = list(filter(lambda item: check_ext(item, "m4a") and check_vcodec(item), formats))
        (best_formats_m4a, vcodec_best) = best_format_id(formats_m4a)
        if best_formats_m4a == "" or best_formats_m4a is None:
            fail_message = f"\033[31m获取信息失败\033[0m, \n错误信息：无法获取音频格式ID"
        else:
            duration_and_id.append(best_formats_m4a)
            if media == "mp4":
                formats_mp4 = list(filter(lambda item: check_resolution(item) and check_ext(item, "mp4") and check_vcodec(item), formats))
                (best_formats_mp4, vcodec_best) = best_format_id(formats_mp4)
                if best_formats_mp4 == ""or best_formats_mp4 is None:
                    fail_message = f"\033[31m获取信息失败\033[0m, \n错误信息：无法获取视频格式ID"
                else:
                    duration_and_id.append(best_formats_mp4)
                    duration_and_id.append(vcodec_best)
    if fail_message is not None:
        return fail_message
    else:
        return duration_and_id



In [11]:
youtube_content_ytid_update = {'UCBR8-60-B28hp2BmDPdntcQ': ['X2676LxDOz8'], 'UCghLs6s95LrBWOdlZUCH4qw': ['fHkNDnPr1N0', 'ril2QorZ6rc', 'hQdlCcyPTv0', 'hOCIz-CMaNA', 'bVFCtn8mQc8', 'vHtdOoXibOo', 'JsvDMh7VMi0', 'AwzGxjsCZeI', 'OL55sKAlWHs', 'nq6qrN1CAh0'], 'UC3elWG-AxVvIF2_xPQb1rkQ': ['6DknTtKwcNw']}
print(video_format("https://www.youtube.com/watch?v=","1vYPOGWuL90","m4a","1080"))

获取信息失败, 
错误信息：无法获取音频格式ID


In [ ]:
import yt_dlp
import pandas as pd
# 获取视频时长模块
def video_duration(video_website, video_url, quality = "480"):
    try:
        # 初始化 yt_dlp 实例, 并忽略错误
        ydl_opts = {
            'ignoreerrors': True,
            'no_warnings': True, 
            #'quiet': True  # 禁止非错误信息的输出
        }
        ydl = yt_dlp.YoutubeDL(ydl_opts)
        # 使用提供的 URL 提取视频信息
        if info_dict := ydl.extract_info(
            f"{video_website}{video_url}", download = False
        ):
            # 获取视频时长并返回
            duration = info_dict.get('duration')
            formats = info_dict.get('formats')
    except Exception as e:
        return None
    print(duration)
    print(formats)
    # 定义条件判断函数
    def check_ext(item, media):
        if "ext" in item:
            return item["ext"] == media
        else:
            return False
    def check_vcodec(item):
        if "vcodec" in item:
            return "vp" not in item["vcodec"].lower()
        else:
            return False
    # 进行筛选
    formats = list(filter(lambda item: check_ext(item, "m4a") and check_vcodec(item), formats))

    for format in formats:
        if "url" in format: del format["url"]
        if "http_headers" in format: del format["http_headers"]
        if "fragments" in format: del format["fragments"]
        if "manifest_url" in format: del format["manifest_url"]
        if "fps" in format: del format["fps"]
        if "vbr" in format: del format["vbr"]
        if "abr" in format: del format["abr"]
        if "audio_ext" in format: del format["audio_ext"]
        if "video_ext" in format: del format["video_ext"]
        if "acodec" in format: del format["acodec"]
        if "tbr" in format: del format["tbr"]
        if "filesize_approx" in format: del format["filesize_approx"]
        if "downloader_options" in format: del format["downloader_options"]
        if "rows" in format: del format["rows"]
        if "columns" in format: del format["columns"]
        if "format_index" in format: del format["format_index"]

    pf=pd.DataFrame(formats)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    print(pf)
    filesize_max = 0
    format_id_max = ""
    for format in formats:
        if "filesize" in format and (isinstance(format["filesize"], float) or isinstance(format["filesize"], int)) and format["filesize"] > filesize_max:
            filesize_max = format["filesize"]
            format_id_max = format["format_id"]
    print(format_id_max)

In [ ]:
video_duration("https://www.youtube.com/watch?v=","qGs_TD3kBVg","1")


In [ ]:
# 定义条件判断函数
def check_age(item):
    if item["gender"] == "male":
        return item["age"] >= 30
    elif item["gender"] == "female":
        return item["age"] >= 25
    else:
        return False  # 非男非女的不符合条件

# 使用 lambda 表达式创建过滤函数
age_filter = lambda item: check_age(item)
country_filter = lambda item: item["country"] == "USA"

# 进行筛选
formats = [
    {"name": "Alice", "gender": "female", "age": 23, "country": "USA"},
    {"name": "Bob", "gender": "male", "age": 31, "country": "Canada"},
    {"name": "Charlie", "gender": "male", "age": 28, "country": "USA"},
    {"name": "David", "gender": "male", "age": 27, "country": "USA"},
    {"name": "Eve", "gender": "female", "age": 26, "country": "USA"}
]

filtered_formats = list(filter(lambda item: age_filter(item) and country_filter(item), formats))

# 输出结果
print(filtered_formats)  # [{'name': 'Charlie', 'gender': 'male', 'age': 28, 'country': 'USA'}, {'name': 'David', 'gender': 'male', 'age': 27, 'country': 'USA'}, {'name': 'Eve', 'gender': 'female', 'age': 26, 'country': 'USA'}]


In [ ]:
def a():
    return 1, 5, 3
a, b, c = a()
print(a, b, c)

In [ ]:
a = 1
while a<100:
    print('\033[{a}mabcd')
    a += 1